# 로컬에서 훈련 하기
- 이 노트북은 로컬 (현재 머신) 에서 Hugging Face Accelerator + PyTorch FSDP 로 파인 튜닝 합니다.

## 1. 환경 셋업

### Hugging Face Token 입력
- [중요] HF Key 가 노출이 안되도록 조심하세요.

In [15]:
import torch, trl

In [17]:
torch.__version__, trl.__version__

('2.4.1+cu121', '0.11.1')

In [18]:
import os

def set_hf_key_env_vars(hf_key_name, key_val):
    os.environ[hf_key_name] = key_val

def get_hf_key_env_vars(hf_key_name):
    HF_key_value = os.environ.get(hf_key_name)

    return HF_key_value


is_sagemaker_notebook = True
if is_sagemaker_notebook:
    hf_key_name = "HF_KEY"
    key_val = "<Type Your HF Key>"
    set_hf_key_env_vars(hf_key_name, key_val)
    HF_TOKEN = get_hf_key_env_vars(hf_key_name)
else: # VS Code
    from dotenv import load_dotenv
    HF_TOKEN = os.getenv('HF_TOKEN')


# Log in to HF
!huggingface-cli login --token {HF_TOKEN}


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ec2-user/.cache/huggingface/token
Login successful


### 저장된 변수 로딩

In [19]:
%store -r data_folder
%store -r train_data_json 
%store -r validation_data_json 
%store -r test_data_json 
%store -r full_train_data_json 
%store -r full_validation_data_json 
%store -r full_test_data_json


print("data_folder: ", data_folder)
print("train_data_json: ", train_data_json)
print("validation_data_json: ", validation_data_json)
print("test_data_json: ", test_data_json)
print("full_train_data_json: ", full_train_data_json)
print("full_validation_data_json: ", full_validation_data_json)
print("full_test_data_json: ", full_test_data_json)

data_folder:  ../data/naver-news-summarization-ko
train_data_json:  ../data/naver-news-summarization-ko/train/train_dataset.json
validation_data_json:  ../data/naver-news-summarization-ko/validation/validation_dataset.json
test_data_json:  ../data/naver-news-summarization-ko/test/test_dataset.json
full_train_data_json:  ../data/naver-news-summarization-ko/full_train/train_dataset.json
full_validation_data_json:  ../data/naver-news-summarization-ko/full_validation/validation_dataset.json
full_test_data_json:  ../data/naver-news-summarization-ko/full_test/test_dataset.json


In [20]:
import torch
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from datasets import Dataset, load_dataset
from datasets import load_dataset
from transformers import set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import warnings
warnings.filterwarnings("ignore")

## 2. 베이스 모델 준비

In [21]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"

In [22]:
prefix = "llama-3-1-8b-qlora-naver-news"
output_dir = f"/home/ec2-user/SageMaker/models/{prefix}"

### Config YAML 파일 생성

In [23]:
%%writefile accelerator_config/local_llama_3_8b_qlora.yaml
# script parameters
model_id: "meta-llama/Meta-Llama-3.1-8B-Instruct"
#model_id: "meta-llama/Llama-3.2-1B-Instruct"
###########################
# small samples for Debug
###########################
train_dataset_path: "../data/naver-news-summarization-ko/train"                      # path to dataset
validation_dataset_path: "../data/naver-news-summarization-ko/validation"                      # path to dataset
#test_dataset_path: "../data/naver-news-summarization-ko/test"                      # path to dataset
per_device_train_batch_size: 1         # batch size per device during training
per_device_eval_batch_size: 1          # batch size for evaluation
gradient_accumulation_steps: 1         # number of steps before performing a backward/update pass
###########################
# large samples for evaluation
###########################
# train_dataset_path: "../data/naver-news-summarization-ko/full_train"                      # path to dataset
# validation_dataset_path: "../data/naver-news-summarization-ko/full_validation"                      # path to dataset
# test_dataset_path: "../data/naver-news-summarization-ko/full_test"                      # path to dataset
# per_device_train_batch_size: 16         # batch size per device during training
# per_device_eval_batch_size: 1          # batch size for evaluation
# gradient_accumulation_steps: 2         # number of steps before performing a backward/update pass
###########################
max_seq_length:  2048              # max sequence length for model and packing of the dataset


# training parameters
output_dir: "/home/ec2-user/SageMaker/models/llama-3-1-8b-fsdp-qlora-naver-news" # Temporary output directory for model checkpoints
report_to: "tensorboard"               # report metrics to tensorboard
logging_dir: "/home/ec2-user/SageMaker/logs/llama-3-1-8b-fsdp-qlora-naver-news" # log folder for tensorboard
learning_rate: 0.0002                  # learning rate 2e-4
lr_scheduler_type: "constant"          # learning rate scheduler
num_train_epochs: 1                    # number of training epochs
optim: adamw_torch                     # use torch adamw optimizer
logging_steps: 10                      # log every 10 steps
save_strategy: epoch                   # save checkpoint every epoch
evaluation_strategy: epoch             # evaluate every epoch
max_grad_norm: 0.3                     # max gradient norm
warmup_ratio: 0.03                     # warmup ratio
bf16: true                             # use bfloat16 precision
tf32: true                             # use tf32 precision
gradient_checkpointing: true          # use gradient checkpointing to save memory

Overwriting accelerator_config/local_llama_3_8b_qlora.yaml


## 3. 훈련 Script 실행

아래는 Hugging Face 의 Accelerator 기반 학습 명령어 입니다.
- 현재 머신에 4개의 GPU 가 있는 경우 입니다. GPU 가 1개 이면 nproc_per_node=1 로 수정해서 실행 하세요. 

```
!torchrun --nproc_per_node=4 \
../../scripts/local_run_qlora.py \
--config config_folder_name/local_llama_3_8b_qlora.yaml
```
- 참고
    - Launching your 🤗 Accelerate scripts, [Link](https://huggingface.co/docs/accelerate/en/basic_tutorials/launch)

In [24]:
import os
config_folder_name = "accelerator_config"
os.makedirs(config_folder_name, exist_ok=True)

### Hugging Face  Accelerator 에 제공할 config.yaml 입니다.

### IMPORTANT!! Set use reentrant to False when we don't use FSDP
```
if training_args.gradient_checkpointing:
    training_args.gradient_checkpointing_kwargs = {"use_reentrant":False**}
```

In [25]:
!CUDA_VISIBLE_DEVICES=0,1,2,3 torchrun --nproc_per_node=4 \
../../scripts/local_run_qlora.py \
--config accelerator_config/local_llama_3_8b_qlora.yaml

[2024-09-28 03:22:35,460] torch.distributed.run: [WARNING] 
[2024-09-28 03:22:35,460] torch.distributed.run: [WARNING] *****************************************
[2024-09-28 03:22:35,460] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2024-09-28 03:22:35,460] torch.distributed.run: [WARNING] *****************************************
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/30_fine_tune/03-fine-tune-llama3/llama3-1/notebook/10-naver-news-QLoRA/../../scripts/local_run_qlora.py", line 8, in <module>
    from trl.commands.cli_utils import  TrlParser
ModuleNotFoundError: No module named 'trl'
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/30_fine_tune/03-fine-tune-llam

## 4. 베이스 모델과 훈련된 모델 머지

In [12]:
print (f'model_id: {model_id}')
print (f'output_dir: {output_dir}')

model_id: meta-llama/Llama-3.1-8B-Instruct
output_dir: /home/ec2-user/SageMaker/models/llama-3-1-8b-qlora-naver-news


### 모델 머지 및 로컬에 저장
- 약 2분 걸림

In [13]:
from peft import AutoPeftModelForCausalLM
import torch

# Load PEFT model on CPU

model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    torch_dtype=torch.float16, ## why bfloat16이 아니지?
    low_cpu_mem_usage=True,
)  
# Merge LoRA and base model and save
merged_model = model.merge_and_unload()
merged_model.save_pretrained(output_dir,safe_serialization=True, max_shard_size="2GB")

ValueError: Can't find 'adapter_config.json' at '/home/ec2-user/SageMaker/models/llama-3-1-8b-qlora-naver-news'

In [ ]:
model.device, merged_model.device

### 머지된 모델 로딩

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer 


# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  pretrained_model_name_or_path = output_dir,
  torch_dtype=torch.float16,
  quantization_config= {"load_in_4bit": True},
  device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

## 5. 추론

### 테스트 데이터 셋 로딩

In [ ]:
from datasets import load_dataset 
from random import randint

def get_message_from_dataset(sample_dataset_json_file):
    # Load our test dataset
    full_test_dataset = load_dataset("json", data_files=sample_dataset_json_file, split="train")

    # Test on sample 
    rand_idx = randint(0, len(full_test_dataset)-1)
    rand_idx = 75
    print("rand_idx: ", rand_idx)
    messages = full_test_dataset[rand_idx]["messages"][:2]
    # messages = test_dataset[rand_idx]["text"][:2]
    print("messages: \n", messages)

    return messages, full_test_dataset, rand_idx

messages, full_test_dataset, rand_idx = get_message_from_dataset(sample_dataset_json_file = full_test_data_json)    

### 추론

In [ ]:

def generate_response(messages, model, tokenizer, full_test_dataset, rand_idx):
    input_ids = tokenizer.apply_chat_template(messages,add_generation_prompt=True,return_tensors="pt").to(model.device)
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id= tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]

    print(f"**Query:**\n{full_test_dataset[rand_idx]['messages'][1]['content']}\n")
    # print(f"**Query:**\n{test_dataset[rand_idx]['text'][1]['content']}\n")
    # print(f"**Original Answer:**\n{test_dataset[rand_idx]['text'][2]['content']}\n")
    print(f"**Original Answer:**\n{full_test_dataset[rand_idx]['messages'][2]['content']}\n")
    print(f"**Generated Answer:**\n{tokenizer.decode(response,skip_special_tokens=True)}")

generate_response(messages, model, tokenizer, full_test_dataset, rand_idx)    



### 할당된 CUDA memory를 Release

In [ ]:
del model
del tokenizer
torch.cuda.empty_cache()